# Online Retail Sales Analysis (Python)

This notebook extends the Power BI analysis by exploring customer behavior,
cancellation impact, and revenue distribution using Python.

The goal is to complement dashboard-level insights with deeper analytical findings.

In [85]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("default")

# Database credentials are omitted for security reasons.

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    dbname="postgres",
    user="data_user",
    password="YOUR_PASSWORD"
)

cur = conn.cursor()
print("Connected to PostgreSQL")

In [87]:
df = pd.read_csv("data/online_retail_II.csv")
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


## Data Cleaning

Basic data cleaning steps are applied to ensure the reliability of downstream analysis.
Transactions without a valid customer identifier are removed, and key numerical and
datetime fields are converted to appropriate data types.

In [89]:
# Remove rows with missing Customer ID
df = df.dropna(subset=["Customer ID"])

# Convert InvoiceDate to datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# Convert Customer ID to int
df["Customer ID"] = df["Customer ID"].astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 824364 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      824364 non-null  object        
 1   StockCode    824364 non-null  object        
 2   Description  824364 non-null  object        
 3   Quantity     824364 non-null  int64         
 4   InvoiceDate  824364 non-null  datetime64[ns]
 5   Price        824364 non-null  float64       
 6   Customer ID  824364 non-null  int64         
 7   Country      824364 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 56.6+ MB


## Feature Engineering

Additional fields are created to support financial analysis and time-based aggregation.
These features mirror the logic used in the Power BI dashboard to ensure consistency
across tools.

In [90]:
# Create Revenue column
df["Revenue"] = df["Quantity"] * df["Price"]

# Create column of Cancelled Transactions
df["IsCancelled"] = df["Invoice"].astype(str).str.startswith("C")

# Create Time features columns
df["Year"] = df["InvoiceDate"].dt.year
df["Month"] = df["InvoiceDate"].dt.month
df["YearMonth"] = df["InvoiceDate"].dt.to_period("M").astype(str)

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue,IsCancelled,Year,Month,YearMonth
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4,False,2009,12,2009-12
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,False,2009,12,2009-12
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,False,2009,12,2009-12
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8,False,2009,12,2009-12
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0,False,2009,12,2009-12


## Database Staging (PostgreSQL)

The cleaned data is loaded into PostgreSQL to enable SQL-based analysis and validation.
The database serves as an intermediate layer between raw data and analytical workflows.

In [91]:
data = df[[
    "Invoice",
    "StockCode",
    "Description",
    "Quantity",
    "InvoiceDate",
    "Price",
    "Customer ID",
    "Country"
]]

In [72]:
insert_query = """
INSERT INTO online_retail (
    invoice,
    stockcode,
    description,
    quantity,
    invoicedate,
    price,
    customer_id,
    country
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""

In [71]:
execute_batch(cur, insert_query, data.values, page_size=10000)
conn.commit()

print("CSV loaded into PostgreSQL")

CSV loaded into PostgreSQL


## SQL-Based Analysis

Key business metrics are computed directly in PostgreSQL using SQL.
This step demonstrates the use of SQL for aggregation, filtering,
and business logic before visualization.

### Total Net Revenue (SQL)

This query computes total revenue excluding cancelled transactions.

In [79]:
total_revenue_sql = """
    SELECT SUM(quantity * price) AS total_revenue
    FROM online_retail
    WHERE invoice NOT LIKE 'C%';
    """

cur.execute(total_revenue_sql)
total_revenue = cur.fetchone()[0]
total_revenue

Decimal('17743429.178')

### Monthly Net Revenue (SQL)

This query computes monthly revenue excluding cancelled transactions,
providing a clean and reliable revenue metric.

In [84]:
monthly_sql = """
SELECT
    DATE_TRUNC('month', invoicedate) AS month,
    SUM(quantity * price) AS net_revenue
FROM online_retail
WHERE invoice NOT LIKE 'C%'
GROUP BY 1
ORDER BY 1;
"""

cur.execute(monthly_sql)
monthly_revenue = cur.fetchall()
monthly_revenue

[(datetime.datetime(2009, 12, 1, 0, 0), Decimal('686654.16')),
 (datetime.datetime(2010, 1, 1, 0, 0), Decimal('557319.062')),
 (datetime.datetime(2010, 2, 1, 0, 0), Decimal('506371.066')),
 (datetime.datetime(2010, 3, 1, 0, 0), Decimal('699608.991')),
 (datetime.datetime(2010, 4, 1, 0, 0), Decimal('594609.192')),
 (datetime.datetime(2010, 5, 1, 0, 0), Decimal('599985.79')),
 (datetime.datetime(2010, 6, 1, 0, 0), Decimal('639066.58')),
 (datetime.datetime(2010, 7, 1, 0, 0), Decimal('591636.74')),
 (datetime.datetime(2010, 8, 1, 0, 0), Decimal('604242.65')),
 (datetime.datetime(2010, 9, 1, 0, 0), Decimal('831615.001')),
 (datetime.datetime(2010, 10, 1, 0, 0), Decimal('1036680.00')),
 (datetime.datetime(2010, 11, 1, 0, 0), Decimal('1172336.042')),
 (datetime.datetime(2010, 12, 1, 0, 0), Decimal('884591.89')),
 (datetime.datetime(2011, 1, 1, 0, 0), Decimal('569445.04')),
 (datetime.datetime(2011, 2, 1, 0, 0), Decimal('447137.35')),
 (datetime.datetime(2011, 3, 1, 0, 0), Decimal('595500.76'

### Yearly Net Revenue (SQL)

In [77]:
yearly_sql = """
    SELECT DATE_TRUNC('year', invoicedate) AS year,
            SUM(quantity * price) AS net_revenue
    FROM online_retail
    WHERE invoice NOT LIKE 'C%'
    GROUP BY 1
    ORDER BY 1;
    """
cur.execute(yearly_sql)
yearly_revenue = cur.fetchall()
yearly_revenue

[(datetime.datetime(2009, 1, 1, 0, 0), Decimal('686654.16')),
 (datetime.datetime(2010, 1, 1, 0, 0), Decimal('8718063.004')),
 (datetime.datetime(2011, 1, 1, 0, 0), Decimal('8338712.014'))]

### Top Customers by Revenue (SQL)

This query identifies the customers generating the highest revenue,
highlighting revenue concentration.

In [80]:
top_customers_sql = """
    SELECT customer_id, 
            SUM(quantity * price) AS revenue
    FROM online_retail
    WHERE invoice NOT LIKE 'C%'
    GROUP BY customer_id
    ORDER BY revenue DESC
    LIMIT 10;
    """
cur.execute(top_customers_sql)
top_customers = cur.fetchall()

top_customers

[(18102, Decimal('608821.65')),
 (14646, Decimal('528602.52')),
 (14156, Decimal('313946.37')),
 (14911, Decimal('295972.63')),
 (17450, Decimal('246973.09')),
 (13694, Decimal('196482.81')),
 (17511, Decimal('175603.55')),
 (16446, Decimal('168472.50')),
 (16684, Decimal('147142.77')),
 (12415, Decimal('144458.37'))]

### Cancellation Rate (SQL)

This query calculates the percentage of cancelled transactions
to assess operational impact.

In [82]:
cancelled_sql = """
    SELECT COUNT(*)FILTER(WHERE invoice LIKE 'C%')::float / COUNT(*) * 100
    FROM online_retail;
    """
cur.execute(cancelled_sql)
cancelled_pct = round(cur.fetchone()[0], 2)

cancelled_pct

2.27

## Key Findings

- Revenue shows seasonal variation across the analyzed period
- Revenue is concentrated among a small number of customers
- Cancelled transactions represent a noticeable share of total activity and are excluded from revenue calculations


## Interpretation

The results highlight seasonal behavior and customer concentration,
which are important factors for revenue analysis and reporting.

## Final Data Model

The main business metrics are calculated in PostgreSQL using SQL views.
These views are used as the main data source and can be connected directly
to BI tools such as Power BI.